## Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

---

## Thu thập dữ liệu

Trong bài này, ta sử dụng thư viện `Beautiful Soup` để cào dữ liệu từ trang [Wolrdometer](https://www.worldometers.info/coronavirus/) vào mỗi **7:00** hằng ngày.

Chú ý rằng, ta được phép sử dụng bộ dữ liệu này cho mục đích học tập.

---

## Khám phá dữ liệu

### Đọc dữ liệu từ file csv vào dataframe

In [2]:
data = pd.read_csv('data/covid_data.csv')
data.head()

,Date,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious Critical,Tot Cases/1M pop,Deaths/1M pop,Total Tests,Tests/1M pop,Population
0,2022-04-21,China,193953,2841.0,4663.0,8.0,157869.0,2185.0,31421.0,223.0,135.0,3.0,160000000.0,111163.0,1.439324e+09
1,2022-04-21,USA,82553058,45204.0,1017609.0,299.0,80355389.0,34592.0,1180060.0,1492.0,246798.0,3042.0,999047824.0,2986727.0,3.344959e+08
2,2022-04-21,India,43050877,903.0,522095.0,33.0,42514479.0,NaN,14303.0,698.0,30654.0,372.0,833377052.0,593395.0,1.404421e+09
3,2022-04-21,Brazil,30330629,18660.0,662556.0,86.0,29353398.0,12596.0,314675.0,8318.0,140890.0,3078.0,63776166.0,296249.0,2.152787e+08
4,2022-04-21,France,28076017,104007.0,144799.0,185.0,25618858.0,148402.0,2312360.0,1677.0,428423.0,2210.0,266484045.0,4066382.0,6.553345e+07


### Dữ liệu gồm bao nhiều dòng, bao nhiêu cột?

In [3]:
num_rows = data.shape[0]
num_cols = data.shape[1]
print('Số dòng: ', num_rows)
print('Số cột: ', num_cols)

Số dòng:  456
Số cột:  15


### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng tương ứng với thông tin số ca nhiễm của một nước trong một ngày. 

Có vẻ như không xảy ra các dòng có ý nghĩa khác nhau

### Dữ liệu có các dòng bị lặp không?

In [4]:
duplicated_rows = data.duplicated().sum()
if duplicated_rows > 0:
    print("Số dòng lặp: ", duplicated_rows)
else:
    print("Không có dòng lặp")

Không có dòng lặp


Dữ liệu được thêm vào mỗi ngày nên tạm thời ta không nhận xét gì thêm ở đây

### Mỗi cột có ý nghĩa gì?

|Tên cột| Mô tả |
|:-:| :--|
|**Date**| Ngày lấy số liệu |
|**Country**| Nước |
|**Total Cases**| tổng số ca nhiễm mới từ trước đến nay |
|**New Cases**| số ca mới ngày hôm nay |
|**Total Deaths**| tổng số ca tử vong từ trước đến nay  |
|**New Deaths**| số ca tử vong mới ngày hôm nay|
|**Total Recovered**| tổng số ca phục hồi |
|**New Recovered**| số ca phục hồi trong ngày|
|**Active Cases**| số ca đang điều trị |
|**Serious Critical**| số ca nguy kịch |
|**Total Cases/1M pop**| số ca nhiễm / 1 triệu dân |
|**Deaths/1M pop**| số ca từ vong/ 1 triệu dân |
|**Total Tests**| tổng số xét nghiệm |
|**Tests/1M pop**| số xét nghiệm/1 triệu dân |
|**Population**| dân số |

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

Giờ ta xem qua kiểu dữ liệu của các cột nhằm phát hiện cột có kiểu dữ liệu chưa phù hợp

In [5]:
data.dtypes

Date                 object
Country              object
Total Cases           int64
New Cases           float64
Total Deaths        float64
New Deaths          float64
Total Recovered     float64
New Recovered       float64
Active Cases        float64
Serious Critical    float64
Tot Cases/1M pop    float64
Deaths/1M pop       float64
Total Tests         float64
Tests/1M pop        float64
Population          float64
dtype: object

Ta cần chuyển cột `Date` sang kiểu dữ liệu `datetime`

---

## Tiền Xử Lý Dữ Liệu

Ở Lab1, ta sẽ chỉ làm trên dữ liệu từ ngày **21/4/2022-28/4/20022**

Ta cần chuyển cột `Date`  về kiểu dữ liệu `datetime`.

Bên cạnh đó, từ dữ liệu gốc, ta lọc những dòng có ngày từ ngày **21/4/2022**  đến **28/4/20022**, lưu vào dataframe `filtered_covid_data`

In [26]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

begin_date = pd.Timestamp('2022-04-21')
end_date = pd.Timestamp('2022-04-28')

filtered_covid_data = data[(data.Date >= begin_date) & (data.Date <= end_date)]
filtered_covid_data.head()

,Date,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious Critical,Tot Cases/1M pop,Deaths/1M pop,Total Tests,Tests/1M pop,Population
0,2022-04-21,China,193953,2841.0,4663.0,8.0,157869.0,2185.0,31421.0,223.0,135.0,3.0,160000000.0,111163.0,1.439324e+09
1,2022-04-21,USA,82553058,45204.0,1017609.0,299.0,80355389.0,34592.0,1180060.0,1492.0,246798.0,3042.0,999047824.0,2986727.0,3.344959e+08
2,2022-04-21,India,43050877,903.0,522095.0,33.0,42514479.0,NaN,14303.0,698.0,30654.0,372.0,833377052.0,593395.0,1.404421e+09
3,2022-04-21,Brazil,30330629,18660.0,662556.0,86.0,29353398.0,12596.0,314675.0,8318.0,140890.0,3078.0,63776166.0,296249.0,2.152787e+08
4,2022-04-21,France,28076017,104007.0,144799.0,185.0,25618858.0,148402.0,2312360.0,1677.0,428423.0,2210.0,266484045.0,4066382.0,6.553345e+07


**Ở các bước tiếp theo ta sẽ khám phá dữ liệu trên tập dữ liệu mới (`filtered_covid_data`)**

---

## Khám Phá Dữ Liệu (tiếp tục)

### Sự phân bố giá trị của mỗi cột

#### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào?

Với các cột có kiểu dữ liệu số, ta tính:

- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu
- Giá trị min
- Giá trị lower quartile (phân vị 25)
- Giá trị median (phân vị 50)
- Giá trị upper quartile (phân vị 75)
- Giá trị max

In [27]:
pd.set_option("max_colwidth", 300)
pd.set_option('display.max_columns', None)

nume_cols = ['Total Cases', 'New Cases', 'Total Deaths', 'New Deaths', 'Total Recovered',
                'New Recovered', 'Active Cases', 'Serious Critical', 'Tot Cases/1M pop',
                'Deaths/1M pop', 'Total Tests', 'Tests/1M pop', 'Population']

index_nume_col =["missing_percentage", "min", "lower_quartile", "median", "upper_quartile", "max"]

data_nume_col= np.array([
    (filtered_covid_data[nume_cols].isna().mean()*100).round(1),
    filtered_covid_data[nume_cols].min().round(1),
    filtered_covid_data[nume_cols].quantile(.25).round(1),
    filtered_covid_data[nume_cols].median(),
    filtered_covid_data[nume_cols].quantile(.75).round(1),
    filtered_covid_data[nume_cols].max().round(1)]
)
nume_col_info_df = pd.DataFrame(data = data_nume_col, columns = nume_cols, index= index_nume_col)
nume_col_info_df

,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious Critical,Tot Cases/1M pop,Deaths/1M pop,Total Tests,Tests/1M pop,Population
missing_percentage,0.0,43.0,3.9,69.3,6.6,53.1,6.6,33.3,0.9,4.8,7.000000e+00,7.0,9.000000e-01
min,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,9.0,2.0,5.117000e+03,5099.0,8.050000e+02
lower_quartile,22581.5,26.8,221.2,2.0,13872.0,26.0,209.8,5.0,11127.0,172.0,3.472968e+05,164770.2,5.576925e+05
median,163079.0,196.5,2201.0,9.0,131100.0,308.5,2651.0,19.5,88852.5,765.0,2.120364e+06,773544.0,5.795836e+06
upper_quartile,1064110.8,1196.8,13990.0,30.2,959396.2,2434.8,30497.2,177.0,236081.8,1844.0,1.224277e+07,2212139.0,2.193454e+07
max,82628089.0,139849.0,1018154.0,646.0,80402401.0,243700.0,2928053.0,8318.0,704474.0,6294.0,1.000126e+09,21809742.0,1.439324e+09


**Nhận xét**:

#### Với mỗi cột có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào?

Với các cột có kiểu dữ liệu dạng `categorical`, ta cần tính:

- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu
- Số lượng các giá trị khác nhau (không xét giá trị thiếu):
- Tỉ lệ % (từ 0 đến 100) của mỗi giá trị được sort theo tỉ lệ % giảm dần (không xét giá trị thiếu)

In [28]:
cate_cols = ['Country']
index_cate_cols = ["missing_percentage", "num_values", "value_percentages"]


def count_value_percentages(data):
    return (data.value_counts()/data.count() *100).to_dict()


data_cate_col= np.array([
    (filtered_covid_data[cate_cols].isna().mean() *100),
    filtered_covid_data[cate_cols].nunique(),
    filtered_covid_data[cate_cols].agg(count_value_percentages)
])

cate_col_info_df = pd.DataFrame(data = data_cate_col, columns = cate_cols, index= index_cate_cols)
cate_col_info_df

,Country
missing_percentage,0.0
num_values,228
value_percentages,"{'Spain': 0.43859649122807015, 'Ukraine': 0.43859649122807015, 'Tonga': 0.43859649122807015, 'Antigua and Barbuda': 0.43859649122807015, 'Netherlands': 0.43859649122807015, 'Ecuador': 0.43859649122807015, 'Kenya': 0.43859649122807015, 'China': 0.43859649122807015, 'Cuba': 0.43859649122807015, 'M..."


**Nhận xét**:

---

## Tiền xử lý dữ liệu

Ta nhận thấy cột `New Cases`, `New Recovered`, `New Deaths` có tỉ lệ thiếu cao. Ta biết được rằng, những ô nào không có giá trị tức là bằng 0 (0 ca nhiễm/hồi phục/chết trong ngàyngày), Vậy nên ta sẽ điền 0 vào các giá trị thiếu này

In [29]:
filtered_covid_data[['New Cases', 'New Recovered',  'New Deaths']] = \
    filtered_covid_data[['New Cases', 'New Recovered',  'New Deaths']].replace(np.nan, 0)

In [30]:
filtered_covid_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456 entries, 0 to 455
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              456 non-null    datetime64[ns]
 1   Country           456 non-null    object        
 2   Total Cases       456 non-null    int64         
 3   New Cases         456 non-null    float64       
 4   Total Deaths      438 non-null    float64       
 5   New Deaths        456 non-null    float64       
 6   Total Recovered   426 non-null    float64       
 7   New Recovered     456 non-null    float64       
 8   Active Cases      426 non-null    float64       
 9   Serious Critical  304 non-null    float64       
 10  Tot Cases/1M pop  452 non-null    float64       
 11  Deaths/1M pop     434 non-null    float64       
 12  Total Tests       424 non-null    float64       
 13  Tests/1M pop      424 non-null    float64       
 14  Population        452 non-

Hiện tại có một số cột đang thiết dữ liệu, những cột đó đang chứa kiểu dữ liệu số, ta tiến hành điền mean vào giá trị thiếu

In [31]:
miss_value_cols = ['Total Deaths', 'Total Recovered', 'Active Cases', 'Serious Critical',
                    'Tot Cases/1M pop', 'Deaths/1M pop', 'Total Tests', 'Tests/1M pop', 'Population']

def fill_mean_value(col):
    mean_value = int(np.mean(col, axis = 0))
    col.replace(np.nan, mean_value, inplace=True)
    return col

filtered_covid_data[miss_value_cols] = filtered_covid_data[miss_value_cols].apply(fill_mean_value)
filtered_covid_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456 entries, 0 to 455
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              456 non-null    datetime64[ns]
 1   Country           456 non-null    object        
 2   Total Cases       456 non-null    int64         
 3   New Cases         456 non-null    float64       
 4   Total Deaths      456 non-null    float64       
 5   New Deaths        456 non-null    float64       
 6   Total Recovered   456 non-null    float64       
 7   New Recovered     456 non-null    float64       
 8   Active Cases      456 non-null    float64       
 9   Serious Critical  456 non-null    float64       
 10  Tot Cases/1M pop  456 non-null    float64       
 11  Deaths/1M pop     456 non-null    float64       
 12  Total Tests       456 non-null    float64       
 13  Tests/1M pop      456 non-null    float64       
 14  Population        456 non-

Sau cùng ta lưu dữ liệu đã qua tiền xử lý vào file `preprocessed_covid_data`

In [32]:
filtered_covid_data.to_csv('data/preprocessed_covid_data.csv')

---

## Đưa ra câu hỏi ý nghĩa cần trả lời